In [17]:
#Importing common libraries
import pandas as pd
import numpy as np
import collections
import copy
import random
import matplotlib.pyplot as plt
from datetime import datetime


#Importing text preprocessing methods:
from nlp.preprocessing import (
    clean_text,
    preprocess,
    tokenize,
    preprocess_document,
    tokenize_document,
    get_stopwords, 
    lemmatization_document,
    get_canonical_words)
from textpp_ptbr.preprocessing import TextPreProcessing as tpp
from gensim.parsing.preprocessing import (
    strip_multiple_whitespaces,
    strip_non_alphanum,
    strip_punctuation2,
    strip_short)

#Importing libraries to check spelling:
from item.spellcheckeropt import SpellcheckerOpt
from item.utils import get_tokens_set


#Importing text analysis:
from nlp.utils import (
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)

#Importing text statistics:
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)

#Importing baseline approaches for clustering:
from nlp.grouping import (
    get_groups,
    get_groups_size,
    get_unigram_groups,
    get_two_tokens_groups,
    get_first_token_groups,
    get_bigram_groups,
    get_first_two_groups,
    groups_frequency_sort
)

#Importing the stucture of the descriptions:
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)

#Importing xmeans through pyclustering library:
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer;
from pyclustering.cluster.xmeans import xmeans

#Importing the HDBSCAN stand-alone method:
import hdbscan
#Importing UMAP dimensionality reduction method:
import umap

#Importing the multiprocessing library:
import multiprocessing

#Importing the libraries to save the final resutls and making it possible to load them:
import nltk
import pickle


#Get the list of words (medicines and nouns) from the list of descriptions
#in a specific group.
def get_list_of_tokens_med_n(group_desc, itemlist, medicines, canonical_form, word_class):
    list_words = list()

    for desc_id in group_desc:
        words = itemlist.items_list[desc_id].get_item_dict()['palavras']
        for p in words:
            if((p not in list_words)): 
                if ((p in medicines) or ((p in word_class) and (word_class[p] == 'N'))):
                    list_words.append(p)
                
    list_words.sort()
    
    return list_words


#Get the list of words (medicines, nouns, verbs, adjectives and numerals) from the list of descriptions
#in a specific group.
def get_list_of_tokens_med_n_v_a_num(group_desc, itemlist, medicines, canonical_form, word_class):
    list_words = list()

    for desc_id in group_desc:
        words = itemlist.items_list[desc_id].get_item_dict()['palavras']
        for p in words:
            if((p not in list_words)): 
                if ((p in medicines) or ((p in word_class) and ((word_class[p] == 'N') or 
                    (word_class[p] == 'A') or (word_class[p] == 'V') or (word_class[p].isnumeric())))):
                    list_words.append(p)
                
    list_words.sort()
    
    return list_words
    
    
#Get the list of words (medicines, nouns, verbs, adjectives and numerals) from the list of descriptions
#in a specific group.
def get_list_of_tokens_med_n_v_a_num_adv(group_desc, itemlist, medicines, canonical_form, word_class):
    list_words = list()

    for desc_id in group_desc:
        words = itemlist.items_list[desc_id].get_item_dict()['palavras']
        for p in words:
            if((p not in list_words)): 
                if ((p in medicines) or ((p in word_class) and ((word_class[p] == 'N') or 
                    (word_class[p] == 'A') or (word_class[p] == 'V') or (word_class[p] == 'DET+Num') or 
                    (word_class[p] == 'ADV') or (word_class[p].isnumeric())))):
                    list_words.append(p)
                
    list_words.sort()
    
    return list_words

    
#Get all list of words in 'palavras' from the list of descriptions from the first token groups.
def get_all_tokens(group_desc, itemlist, medicines, canonical_form, word_class):
    list_words = list()

    for desc_id in group_desc:
        words = itemlist.items_list[desc_id].get_item_dict()['palavras']
        for p in words:
            list_words.append(p)
                
    list_words.sort()
    
    return list_words

#Define a zero matrix based on the size of the number 
#of descriptions in that group (row) and the number of 
#words (only medicines and nouns) from all descriptions
#in that group.
def define_zero_matrix(group_desc, itemlist, medicines, canonical_form, word_class, tokens):
    if(tokens == 0):
        print('BOW composed only by nouns and medicines/medical terms of the descriptions!')        
        list_words = get_list_of_tokens_med_n(group_desc, itemlist, medicines, canonical_form, word_class)
    elif(tokens == 1):
        print('BOW composed by all words of the descriptions!')
        list_words = get_all_list_of_words(group_desc, itemlist, medicines, canonical_form, word_class)
    elif(tokens == 2):
        print('BOW composed only by nouns, adjectives, numerals, verbs and medicines/medical terms of the descriptions!')
        list_words = get_list_of_tokens_med_n_v_a_num(group_desc, itemlist, medicines, canonical_form, word_class)
    elif(tokens == 3):
        print('BOW composed only by nouns, adjectives, numerals, adverbs, verbs and medicines/medical terms of the descriptions!')
        list_words = get_list_of_tokens_med_n_v_a_num_adv(group_desc, itemlist, medicines, canonical_form, word_class)               
    else:
        print('Option not available! Using all words of the descriptions to make the bag...')
        list_words = get_all_tokens(group_desc, itemlist, medicines, canonical_form, word_class)


    rows = len(group_desc)
    columns = len(list_words)
    matrix_bow = np.zeros((rows, columns))
    print('Rows = ' + str(rows))    
    #print('Columns = ' + str(columns))
    return matrix_bow, list_words, rows, columns

# Define the bag-of-words matrix.
def define_description_bow(group_desc, itemlist, medicines, canonical_form, word_class, tokens):
    matrix_list = define_zero_matrix(group_desc, itemlist, medicines, canonical_form, word_class, tokens)
    zeros = matrix_list[0]
    list_words = matrix_list[1]
    rows = matrix_list[2]
    columns = matrix_list[3]
    preproc_descs = np.empty((rows, 1), dtype='object')
    original_descs = np.empty((rows, 1), dtype='object')
    ids_descs = np.empty((rows, 1), dtype='object')
    
    i = 0
    for desc_id in group_desc:
        words = itemlist.items_list[desc_id].get_item_dict()['palavras']        
        desc_original = itemlist.items_list[desc_id].get_item_dict()['original']
        desc_prep = itemlist.items_list[desc_id].get_item_dict()['original_prep']
        desc_prep_rep = str(desc_prep).replace('\'', '').replace('[', '').replace(']', '').replace(',', '')
        
        original_descs[i, 0] = desc_original
        preproc_descs[i, 0 ] = desc_prep_rep
        ids_descs[i, 0 ] = desc_id
        
        for w in words:
            if(w in list_words):
                k = list_words.index(w)
                zeros[i, k]  = 1.0
        i = i + 1       
        
    result_descs = np.concatenate((original_descs, preproc_descs), axis=1)
    result_descs_w_ids = np.concatenate((ids_descs, result_descs), axis=1)
    
    return zeros, rows, columns, result_descs_w_ids

#It applies x-means on the bag of words.
def cluster_by_xmeans(bow, number_of_descriptions):
    cluster_size_limit = round(number_of_descriptions/30)
    xmeans_instance = xmeans(bow, kmax=cluster_size_limit, ccore=False)
    xmeans_instance.process();
    clusters = xmeans_instance.get_clusters();
    
    return clusters

#It just transfors the sklearn output to the pyclustering output
#as they differ in terms of representation.
def transform_sklearn_to_pyclustering(output):
    output_dict = {}
    i = 0
    
    while i < len(output):     
        if(output[i] not in output_dict):
            aux_arr = []
            aux_arr.append(i)
            output_dict[output[i]] = aux_arr
        else:
            aux_arr = output_dict[output[i]]
            aux_arr.append(i)
            output_dict[output[i]] = aux_arr       
        
        i = i + 1
        
    output_arr = []
    negative_key = -1
    
    j = 0
    for key in output_dict:
        if(key == -1):
            #print('found it at ' + str(j))
            negative_key = j

        j = j + 1
        output_arr.append(output_dict[key])
    
    return output_arr, negative_key


#It applies hdbscan on the bag of words.
def cluster_by_hdbscan(bow, employed_metric, groups_ft):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=30, metric=employed_metric, min_samples=1)
    clusters_sklearn = clusterer.fit_predict(bow)

    clusters_pyclustering = transform_sklearn_to_pyclustering(clusters_sklearn)

    return clusters_pyclustering, clusters_sklearn
    

def check_hdbscan_metrics(metric):
    available_metrics = ['braycurtis','canberra','chebyshev','cityblock','dice','euclidean',
    'hamming','haversine','infinity','jaccard','kulsinski','l1','l2','mahalanobis','manhattan',
    'matching','minkowski','p','pyfunc','rogerstanimoto','russellrao','seuclidean','sokalmichener',
    'sokalsneath','wminkowski']

    employed_metric = metric

    #It hecks if the employed metric is available for HDBSCAN;
    #If it is not available, it employs l2
    if(employed_metric not in available_metrics):
        employed_metric = 'l2'

    return employed_metric


#It calls the specific method depending on 'cluster_alg' parameter.
#groups_ft is used when we have outliers, so we can separate these outliers
#considering the groups (from the First Token approach) they actually represent.
def general_clustering(bow, groups_ft, number_of_descriptions, cluster_alg, cluster_alg_metric):
    clusters = None
    employed_metric = check_hdbscan_metrics(cluster_alg_metric)
    
    if(cluster_alg == 'xmeans'):
        print('Clustering by X-Means.')
        clusters = cluster_by_xmeans(bow, number_of_descriptions)
        return clusters, None, None
    #cluster_alg = 1, it employs  HDBSCAN with the Euclidean distance (normalized by l2):
    elif(cluster_alg == 'hdbscan'):
        print('Clustering by HDBSCAN with ' + employed_metric +  ' metric.')
        clusters = cluster_by_hdbscan(bow, employed_metric , groups_ft)
        return clusters[0][0], clusters[0][1], clusters[1]
    #otherwise,  it employs HDBSCAN with an Euclidean distance (normalized by l2):
    else:
        print('Option not available. Employing HDBSCAN clustering algorithm with l2 metric.')
        clusters = cluster_by_hdbscan(bow, 'l2', groups_ft)
        return clusters[0][0], clusters[0][1], clusters[1]




#Translate the generated ids of the clustering approach to actual description ids.
def translate_id_to_descriptions(ids, descriptions_ids):
    arr = []
    
    for i in ids:
        arr.append(descriptions_ids[i])
    return arr

def check_umap_metrics(metric):
    available_metrics = ['euclidean','manhattan','chebyshev','minkowski','canberra','braycurtis',
    'mahalanobis','wminkowski','seuclidean','cosine','correlation','haversine','hamming',
    'jaccard','dice','russelrao','kulsinski','ll_dirichlet','hellinger','rogerstanimoto',
    'sokalmichener','sokalsneath','yule']

    employed_metric = metric

    #It hecks if the employed metric is available for UMap;
    #If it is not available, it employs cosine
    if(employed_metric not in available_metrics):
        employed_metric = 'cosine'

    return employed_metric

#Dimensionality reduction to the bag-of-words.
def dimensionality_reduction(bow, dr_alg, dr_n_comp, dr_metric):
    flag = False
    bow_reduced = None    

    if(dr_alg == 'umap'):
        if(bow[1] > 0 and bow[2] > 0):
            try:
                employed_metric = check_umap_metrics(dr_metric)
                bow_reduced = umap.UMAP(n_components=dr_n_comp, metric=employed_metric, low_memory = True).fit_transform(bow[0])
                print('UMAP employed')                
            except:
                print('#####Exception occurred')
                bow_reduced = bow[0]
                flag = True
    else:
        print('No dimensionality reduction employed. Using the traditional bag of words.')
        bow_reduced = bow[0]
        
    rows, cols = bow_reduced.shape
        
    return bow_reduced, rows, cols
        


#It clusters again the groups generated by the first token approach. For now, this method only accepts X-Means and HDBScan with specific characteristics.
def cluster_on_first_token_groups_bow(first_token_groups, itemlist, it_thread, lower, upper, 
                                      medicines, canonical_form, word_class, tokens,
                                      cluster_alg, cluster_alg_metric, 
                                      dr_alg, dr_n_comp, dr_metric, 
                                      Return_Dict, Return_Repres_Comp):

    print(it_thread)
    #It creates a list of the the keys of these groups:
    groups = list(first_token_groups.keys())
    #It gets the values of each group (i.e., the id of the descriptions into that group):
    group_descriptions = list(first_token_groups.values())
    #It defines the dictionary that will have the clustering with first token
    #together with traditional clustering methods considering a bag-of-words of the descriptions 
    #grouped by the first token approach:
    first_token_plus_bow_traditional_clustering = {}
    #Iterator of the first token groups:
    ft_it = lower
    start_it = lower
    bow_w_dr = None
    df_results = None
    
    while ft_it <= upper:
        print(str(it_thread) + ': ' + str(start_it) + '/' + str(ft_it) + '/' + str(upper))
        #It only considers to cluster again if the number of descritptions of that group has more than 30 descriptions
        if(len(group_descriptions[ft_it]) > 30):
            
            #Bag of words for the group 0:
            bow_raw = define_description_bow(group_descriptions[ft_it], itemlist, medicines, canonical_form, word_class, tokens)
            bow_w_dr = dimensionality_reduction(bow_raw, dr_alg, dr_n_comp, dr_metric)

            #It only applies the traditional clustering methods if the number of rows and columns of the bow are greater than zero:
            if(bow_w_dr[1] > 0 and bow_w_dr[2] > 0):
                #It applies the clusters on the bow of the descriptions:
                clusters_bow_result = general_clustering(bow_w_dr[0], groups[ft_it], len(group_descriptions[ft_it]), cluster_alg, cluster_alg_metric)
                clusters_bow = clusters_bow_result[0]
                negative_index = clusters_bow_result[1]

                if(bow_w_dr[2]==dr_n_comp):
                    first_token = np.full((bow_w_dr[1], 1), groups[ft_it], dtype='object')
                    result_descs = np.concatenate((first_token, bow_raw[3]), axis=1)
                    result_descs_w_dr = np.concatenate((result_descs, bow_w_dr[0]), axis=1)
                    ft_plus_clusters = np.concatenate((first_token, np.c_[clusters_bow_result[2]]), axis=1)
                    ft_plus_clusters_merged = np.c_[["".join(i) for i in ft_plus_clusters[:,0:].astype(str)]]                    
                    result_descs_w_clusters = np.concatenate((result_descs_w_dr, ft_plus_clusters_merged), axis=1)
                    #print(result_descs_w_clusters)
                    if(df_results is None):
                        df_results = result_descs_w_clusters
                    else:
                        df_results = np.concatenate((df_results, result_descs_w_clusters), axis=0)

                it = 0
                for c in clusters_bow:
                    #It translates ids from traditional clustering to actual descriptions (new groups):
                    desc_ids = translate_id_to_descriptions(c, group_descriptions[ft_it])
                    #It defines the key of the map:
                    if(it != negative_index):
                        new_key = groups[ft_it] + '_' + str(it)
                    else:
                        new_key = groups[ft_it] + '_-1'
                        #It sets the maps:
                    first_token_plus_bow_traditional_clustering[new_key] = desc_ids                   
                    it = it + 1     
            else:
                #It returns the first token groups if it is not possible to apply the traditional clustering.
                first_token_plus_bow_traditional_clustering[groups[ft_it]] = group_descriptions[ft_it]
        else:
            #It returns the first token groups if it is not possible to apply the traditional clustering.
            first_token_plus_bow_traditional_clustering[groups[ft_it]] = group_descriptions[ft_it]
        

        ft_it = ft_it + 1
    
    #Returning dictionaries for this process
    Return_Repres_Comp[it_thread] = df_results        
    Return_Dict[it_thread] = first_token_plus_bow_traditional_clustering


#It gets the ranges of the clusters generated by the First Token approach
#This is done in order to the processes work on.
def get_ranges(group_len, n_processes):
    if(n_processes == 1):
        return 0, (group_len - 1)

    total_len = group_len
    num_processes = n_processes
    lower = []
    upper = []
    step = int(total_len/num_processes)

    for k in range(num_processes):
        lower.append(0)
        upper.append(0)

    lower[0] = 0
    upper[0] = step
    i = 1
    j = 0
    while (i < num_processes):    
        upper[i]  = upper[j] + step
        lower[i]  = upper[j] +  1
        if(i%2 != 0):
            upper[i] = upper[i] + 1
        
        i = i + 1
        j = j + 1
    
    #Please, check if the final cluster range ends with 18,034 clusters
    #(i.e., the number of clusters generated by First Token).
    #Depending of the number of processes, you may have to change this "-1"
    #for something else.
    upper[n_processes - 1] = upper[n_processes - 1] - 1 
    return lower, upper

In [10]:
manager_results = multiprocessing.Manager()
Return_Dict = manager_results.dict()

manager_repres = multiprocessing.Manager()
Return_Repres_Comp = manager_repres.dict()   

jobs = []
n_threads = 1

#It loads the medical terms (medicines, drugs, etc):
medicines = get_tokens_set('../dados/palavras/medications.txt')
#It loads the canonical forms and their classes
canonical_form, word_class = get_canonical_words()
print("Read Canonical terms.")
        
#It loads the items from the list:
itemlist = ItemList()
itemlist.load_items_from_file('../dados/items_preprocessed_sp0_sc1.zip', original=True)
                
print('Read data preprocessed')    
#It gets the first tokens of each description and groups
#based on this approach:
first_token_groups = itemlist.get_first_token_groups()
group_len = len(first_token_groups)
first_token_groups_new = {}

#It shuffles the itens based on their keys:
keys_ft = list(first_token_groups.keys())
random.shuffle(keys_ft)
random.shuffle(keys_ft)
    
#It fills another dictionary with the shuffled keys:
for k in keys_ft:
    first_token_groups_new[k] = first_token_groups[k]
    

#It defines the ranges (of the groups) the processes will work on:
thread_ranges = get_ranges(group_len, 10)
print('Read ranges')
print(thread_ranges) 

Read Canonical terms.
Read data preprocessed
Read ranges
([0, 1381, 2762, 4142, 5523, 6903, 8284, 9664, 11045, 12425], [1380, 2761, 4141, 5522, 6902, 8283, 9663, 11044, 12424, 13804])


In [18]:
p = multiprocessing.Process(target=cluster_on_first_token_groups_bow, 
                            args=(first_token_groups_new, itemlist, 0, thread_ranges[0][0], 
                                  thread_ranges[1][0], medicines, canonical_form, word_class, 
                                  0, 'hdbscan', 'l2', 
                                  'umap', 15, 'cosine',
                                  Return_Dict, Return_Repres_Comp))


jobs.append(p)
p.start()
        
#It joins the results
for i in range(n_threads):   
    jobs[i].join()

0
0: 0/0/1380
0: 0/1/1380
0: 0/2/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['panonico' 12167
  'Panotico rapido, Conjunto para coloracao rapida em hematologia. O kit e composto por tres corantes: Rapido 1:agente fixador, Rapido 2: solucao corante'
  'panonico rapido conjunto para coloracao rapido hematologia kit composto tres corante 1 agente fixador 2 solucao corante'
  14.835219383239746 6.561183929443359 -12.502057075500488
  19.035358428955078 6.168514728546143 15.142071723937988
  5.546876430511475 -3.7879538536071777 1.4984290599822998
  -4.567113399505615 16.542892456054688 12.217392921447754
  1.016654133796692 11.11926555633545 2.8754892349243164 'panonico-1']
 ['panonico' 38857 'PANÓTICO - 3X500 ML' 'panonico 3 x 500 ml'
  13.857443809509277 6.243274211883545 -13.020267486572266
  18.501798629760742 6.056699752807617 14.973553657531738
  5.268158912658691 -3.941098213195801 2.226543426

  1.2143986225128174 11.43726634979248 3.2818427085876465 'panonico-1']]
0: 0/3/1380
0: 0/4/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['resultado' 30454 'RESULTADO DE EXAME C/100FLS FRENTE E VERSO 33X24CM'
  ... 4.456676006317139 9.22832202911377 'resultado-1']
 ['resultado' 54192 'RESULTADO DE EXAME DE FEZES' ... 6.941014289855957
  7.334301948547363 'resultado-1']
 ['resultado' 65879 'RESULTADO DE EXAME FESES C/100 FOLHAS' ...
  7.1768293380737305 7.482198715209961 'resultado-1']
 ...
 ['resultado' 1457491
  'RESULTADO DA MAMOGRAFIA, (PROGRAMA NACIONAL DE CONTROLE DO CANCER DO COLO DE ÚTERO E DE MAMA - COR AZUL.,BLOCO COM 100 UNIDADES'
  ... 7.16309928894043 7.148916244506836 'resultado-1']
 ['resultado' 1467742 'RESULTADO DE EXAMES LABORATOR' ...
  6.510304927825928 6.771862030029297 'resultado-1']
 ['resultado' 1479777 'RESULTADO DE EXAMES DE FEZES 01 VIA F8' ...
  6.913076877593994 7.238456

  11.865950584411621 'micro-usb-1']]
0: 0/22/1380
0: 0/23/1380
0: 0/24/1380
0: 0/25/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['cha' 4687 'Chá de Limão-100g. Caixa contendo 15(quinze) saquinhos' ...
  1.0423141717910767 1.6062290668487549 'cha6']
 ['cha' 13326 'CHÁ MATTE EM CAIXA COM 250 GRAMAS' ... 8.885299682617188
  5.892728805541992 'cha5']
 ['cha' 13779
  '(00000572)Chá de erva cidreira, cx contendo 10 saches de 20gr' ...
  0.8056888580322266 1.527966856956482 'cha2']
 ...
 ['cha' 1486147 'CHÁ DE CANELA EMBALAGEM C/20 SACHÊ' ...
  2.7389261722564697 1.7004363536834717 'cha7']
 ['cha' 1489252
  'CHA DE ERVA DOCE - CHA DE ERVA DOCE EM SACHE 1 GRAMA, CAIXA COM 10 UNIDADES'
  ... 6.220381259918213 7.134146213531494 'cha3']
 ['cha' 1489275
  'CHA MATE 100% PURO, EM SACHE DE 1,5 GRAMAS - CHA MATE 100% PURO, EM SACHE DE 1,5 GRAMAS - CAIXA COM 15 UNIDADES'
  ... 2.3784656524658203 1.475308537483215

  4.8940887451171875 'esteira-1']]
0: 0/71/1380
0: 0/72/1380
0: 0/73/1380
0: 0/74/1380
0: 0/75/1380
0: 0/76/1380
0: 0/77/1380
0: 0/78/1380
0: 0/79/1380
0: 0/80/1380
0: 0/81/1380
0: 0/82/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['esomeprazol' 2653 'ESOMEPRASOL 40MG...' ... 2.753852367401123
  6.097195625305176 'esomeprazol0']
 ['esomeprazol' 15233 'ESOMEPRAZOL MAGNESIO 40 MG - CX C/ 28 cpr.' ...
  -1.2988982200622559 8.657147407531738 'esomeprazol1']
 ['esomeprazol' 38712 'ESOMEPRAZOL 20MG C/ 14 COMP.' ...
  3.0308127403259277 6.3055739402771 'esomeprazol0']
 ...
 ['esomeprazol' 1444151 'ESOMEPRAZOL MAGNESIO 20MG - CX C/ 28 Comp.' ...
  -1.3009966611862183 8.89163875579834 'esomeprazol1']
 ['esomeprazol' 1446063 'ESOMEPRASOL 20MG' ... 2.772287607192993
  6.251501083374023 'esomeprazol0']
 ['esomeprazol' 1465945 'ESOMEPRAZOL40MG' ... 2.78525972366333
  6.4178338050842285 'esomeprazol0']]
0: 0/83/

  5.929913520812988 0.4952240586280823 -0.276028573513031 'passar-1']]
0: 0/89/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['olmesartana' 29275
  '0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000990003 OLMESARTANA + ANLODIPINO 40MG + 5MG'
  ... 4.455138683319092 12.631556510925293 'olmesartana0']
 ['olmesartana' 32512
  'Olmesartana medoxomila 40 mg + Hidroclorotiazida 25 mg comp. (Ref. Olmetec)'
  ... -3.476170778274536 1.669655442237854 'olmesartana1']
 ['olmesartana' 77876 'OLMESARTANA + ANLODIPINO 40mg + 10 mg' ...
  4.474453926086426 12.651716232299805 'olmesartana0']
 ...
 ['olmesartana' 1475726 'OLMESARTANA MEDOXOMILA 40MG' ...
  4.229830741882324 12.517619132995605 'olmesartana0']
 ['olmesartana' 1482126
  'OLMESARTANA MEDOXOMILA ASSOCIADA COM HIDROCLOROTIAZIDA 40MG + 25 MG COMPRIMIDO OLMETEC HCT OU SIMILARES'
  ... -3.2357215881347656

...

  -0.8675082921981812 'diacereina-1']]
0: 0/1301/1380
0: 0/1302/1380
0: 0/1303/1380
0: 0/1304/1380
0: 0/1305/1380
0: 0/1306/1380
0: 0/1307/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['cafeina' 59871
  'Cafeina 30 mg + Carisoprol 125 mg + Diclofenaco 50 mg + Paracetamol 300 mg, (comprimidos)'
  'cafeina 30 mg carisoprodol 125 diclofenaco 50 paracetamol 300 comprimido'
  2.613330125808716 2.269742727279663 2.0868613719940186 8.57835578918457
  7.427464008331299 -1.8673685789108276 6.522615909576416
  6.661154270172119 8.486543655395508 0.605304479598999
  11.476841926574707 6.927743911743164 4.283863067626953
  17.388717651367188 2.890990734100342 'cafeina-1']
 ['cafeina' 70174 'CAFEINA+CARISOP+DICLOF.SODIO+'
  'cafeina arison diclo sodio' 3.981304407119751 2.198744535446167
  1.9101054668426514 8.702224731445312 7.966429233551025
  -1.79916250705719 7.23455810546875 6.725832462310791 8.27030181884

  16.873390197753906 3.067030906677246 'cafeina-1']]
0: 0/1308/1380
0: 0/1309/1380
0: 0/1310/1380
0: 0/1311/1380
0: 0/1312/1380
0: 0/1313/1380
0: 0/1314/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['tinidazol' 19544 'TINIDAZOL+NITRATO DE MICONAZOL' ...
  7.740622520446777 5.737598419189453 'tinidazol1']
 ['tinidazol' 23783 'TINIDAZOL  500MG COMP.' ... 4.339291572570801
  7.907022476196289 'tinidazol0']
 ['tinidazol' 25211 'TINIDAZOL 500 mg COMPRIMIDO' ... 4.665114402770996
  7.684550762176514 'tinidazol0']
 ...
 ['tinidazol' 1377031 'TINIDAZOL 500 MG' ... 4.518737316131592
  7.692694187164307 'tinidazol0']
 ['tinidazol' 1463481 'TINIDAZOL + NITRATO DE MICONA' ...
  7.643012046813965 5.994508266448975 'tinidazol1']
 ['tinidazol' 1479911 'TINIDAZOL+MICONAZOL 40GR POMADA' ...
  7.009921550750732 5.724215030670166 'tinidazol1']]
0: 0/1315/1380
0: 0/1316/1380
0: 0/1317/1380
0: 0/1318/1380
0: 0/1319/138

  2.8603103160858154 14.763425827026367 13.42354965209961 'cabide-1']]
0: 0/1321/1380
0: 0/1322/1380
0: 0/1323/1380
0: 0/1324/1380
0: 0/1325/1380
0: 0/1326/1380
0: 0/1327/1380
0: 0/1328/1380
BOW composed only by nouns and medicines/medical terms of the descriptions!
UMAP employed
Clustering by HDBSCAN with l2 metric.
[['paramonoclorofenol' 1263
  'Paramonoclorofenol, canforado, para desinfecção de sistema de canais radiculares. Frasco com 20 ml.'
  ... 5.707361698150635 3.887085199356079 'paramonoclorofenol2']
 ['paramonoclorofenol' 4305 'Paramonoclorofenol' ... 9.672825813293457
  6.265803337097168 'paramonoclorofenol0']
 ['paramonoclorofenol' 6305 'PARAMONOCLOROFENOL 20 ML' ...
  9.733607292175293 6.275102138519287 'paramonoclorofenol0']
 ...
 ['paramonoclorofenol' 1483542 'Paramonoclorofenol, Canforado 20 ML' ...
  -0.6273415684700012 7.902161121368408 'paramonoclorofenol1']
 ['paramonoclorofenol' 1487637 'Paramonoclorofenol, frasco 20ml' ...
  5.911252498626709 4.120553970336914 'p

  -2.625730037689209 -1.3806090354919434 'anilhar-1']]
0: 0/1380/1380


Process Process-12:
Traceback (most recent call last):
  File "/dados01/workspace/ufmg.f01dcc/buildout.python/parts/opt/lib/python3.7/multiprocessing/managers.py", line 811, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/dados01/workspace/ufmg.f01dcc/buildout.python/parts/opt/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/dados01/workspace/ufmg.f01dcc/buildout.python/parts/opt/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-17-ce7ef4e96b05>", line 429, in cluster_on_first_token_groups_bow
    Return_Repres_Comp[it_thread] = df_results
  File "<string>", line 2, in __setitem__
  File "/dados01/workspace/ufmg.f01dcc/buildout.python/parts/opt/lib/python3.7/multiprocessing/managers.py", 

In [ ]:
def main(saving_dict, cluster_option, number_threads, tokens, dim_red):


    now = datetime.now()
    print("Start Time = " + str(now))
    
    manager_results = multiprocessing.Manager()
    Return_Dict = manager_results.dict()

    manager_repres = multiprocessing.Manager()
    Return_Repres = manager_repres.dict()    

    Return_FT_Repres = 
    jobs = []
    n_threads = number_threads

    #It loads the medical terms (medicines, drugs, etc):
    medicines = get_tokens_set('../dados/palavras/medications.txt')
    #It loads the canonical forms and their classes
    canonical_form, word_class = get_canonical_words()
    print("Read Canonical terms.")
        
    #It loads the items from the list:
    itemlist = ItemList()
    itemlist.load_items_from_file('../dados/items_preprocessed_sp0_sc1.zip', original=True)
                
    print('Read data preprocessed')    
    #It gets the first tokens of each description and groups
    #based on this approach:
    first_token_groups = itemlist.get_first_token_groups()
    group_len = len(first_token_groups)
    first_token_groups_new = {}

    #It shuffles the itens based on their keys:
    keys_ft = list(first_token_groups.keys())
    random.shuffle(keys_ft)
    random.shuffle(keys_ft)
    
    #It fills another dictionary with the shuffled keys:
    for k in keys_ft:
        first_token_groups_new[k] = first_token_groups[k]
    
    #It defines the ranges (of the groups) the processes will work on:
    thread_ranges = get_ranges(group_len, n_threads)
    print('Read ranges')
    print(thread_ranges) 
    
    #It creates the processes (balanced by shuffling the keys of the dictionary:
    for i in range(n_threads):
        p = multiprocessing.Process(target=cluster_on_first_token_groups_bow, 
                                    args=(first_token_groups_new, itemlist, i, thread_ranges[0][i], 
                                          thread_ranges[1][i], medicines, canonical_form, word_class, 
                                          cluster_option, tokens, dim_red, Return_dict, Return_Repres))
        jobs.append(p)
        p.start()
        
    #It joins the results
    for i in range(n_threads):   
        jobs[i].join()
        
    dictionary_clusters = {}   
    #It joins the results
    for k in Return_dict:
        print('key: '+str(k))
        dictionary_clusters.update(Return_dict[k])

    #It saves the dictionary in a file, which is possible to reconstruct the final dictionary:
    a_file = open(saving_dict, "wb")
    pickle.dump(dictionary_clusters, a_file)
    a_file.close()
    
    now = datetime.now()
    print("End Time = " + str(now))    